# Monotonically decreasing SoH
The goal of this notebook is to study the results that has been put in the database.

## Setup

### Imports

In [ ]:
import plotly.express as px

from core.pandas_utils import *
from core.stats_utils import *
from transform.results.config import *
from transform.results.main import get_processed_results

### Data extraction

In [ ]:
results = get_processed_results("mercedes-benz")

### Visualization
Let's visualize the results

In [ ]:
px.scatter(results, 
           x="odometer", 
           y="soh", 
           color="model", 
           hover_data=["vin"],
           trendline="ols",
           title="Original SoH results")

In [ ]:
results.head()

In [ ]:
# Option 1 : nunique()
n_vins = results.query("soh != 'nan'")['vin'].nunique()

print(n_vins)

### Demo

In [ ]:
def make_soh_presentable(df:DF) -> DF:
    outliser_mask = mask_out_outliers_by_interquartile_range(df["soh"])
    df = df[outliser_mask].copy()
    if len(df) > 0:
        df["soh"] = force_monotonic_decrease(df["soh"])
    return df

In [ ]:
from logging import getLogger

logger = getLogger("eda.results.monotonically_decreasing_soh")

# Application du traitement sur le DataFrame original
ts_processed = (
    results
    # 3. Appliquer le traitement de SoH par véhicule
    .groupby('vin')
    .apply(make_soh_presentable, include_groups=False)
    .reset_index(drop=False)  # Supprime l'index 'vin' créé par le groupby
    # 1. D'abord, arrondir les dates à la semaine 
    .assign(date=lambda df: df["date"].dt.floor(UPDATE_FREQUENCY))
    # 2. Grouper par vin et date pour avoir une valeur hebdomadaire
    .groupby(["vin", "date"])
    .agg({
        "odometer": "last",    
        "soh": "median",
        "model": "first",
    })
    .pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS, logger=logger)
    .reset_index()
    # 4. Reset final de l'index
    .reset_index(drop=True)
)

In [ ]:
px.line(ts_processed.sort_values(["vin", "odometer"]).dropna(subset=["soh", "odometer"]), x="odometer", y="soh", color="vin", title="Processed SoH results")